## Task 1: Understand the training code 

In [13]:
import pandas as pd

train = pd.read_parquet('data/training_data.parquet')
test = pd.read_parquet('data/testing_data.parquet')

In [14]:
# Split train and test data into features X and targets Y.
target_column_name = 'readmit_status'
Y_train = train[target_column_name]
X_train = train.drop([target_column_name], axis = 1)  
Y_test = test[target_column_name]
X_test = test.drop([target_column_name], axis = 1)  

In [15]:
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import numpy as np

# Transform string data to numeric one-hot vectors
categorical_selector = selector(dtype_exclude=np.number)
categorical_columns = categorical_selector(X_train)
categorial_encoder = OneHotEncoder(handle_unknown="ignore")

# Standardize numeric data by removing the mean and scaling to unit variance
numerical_selector = selector(dtype_include=np.number)
numerical_columns = numerical_selector(X_train)
numerical_encoder = StandardScaler()

# Create a preprocessor that will preprocess both numeric and categorical data
preprocessor = ColumnTransformer([
('categorical-encoder', categorial_encoder, categorical_columns),
('standard_scaler', numerical_encoder, numerical_columns)])

clf = make_pipeline(preprocessor, LogisticRegression(max_iter=1000))

print("Training model...") 
model = clf.fit(X_train, Y_train)
print("Accuracy score: ", clf.score(X_test,Y_test))

Training model...
Accuracy score:  0.8390804597701149


## Task 2: Create a cloud client

In [16]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /home/bstollnitz/git/config.json


## Task 3: Register the training and test data

In [17]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

TRAIN_DATA_NAME = 'hospital_train_parquet'
TEST_DATA_NAME = 'hospital_test_parquet'

training_data = Data(
    name=TRAIN_DATA_NAME,
    path='data/training_data.parquet',
    type=AssetTypes.URI_FILE,
    description='RAI hospital train data'
)
tr_data = ml_client.data.create_or_update(training_data)

test_data = Data(
    name=TEST_DATA_NAME,
    path='data/testing_data.parquet',
    type=AssetTypes.URI_FILE,
    description='RAI hospital test data'
)
ts_data = ml_client.data.create_or_update(test_data)

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	VisualStudioCodeCredential: Failed to get Azure user details from Visual Studio Code.
	AzureCliCredential: ERROR: The command failed with an unexpected error. Here is the traceback:
ERROR: 0. The ID token is not yet valid. Make sure your computer's time and time zone are both correct. Current epoch = 1683147857.  The id_token was: {
  "aud": "04b07795-8ddb-461a-bbee-02f9e1bf7b46",
  "iss": "https://login.microsoftonline.com/

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	VisualStudioCodeCredential: Failed to get Azure user details from Visual Studio Code.
	AzureCliCredential: ERROR: The command failed with an unexpected error. Here is the traceback:
ERROR: 0. The ID token is not yet valid. Make sure your computer's time and time zone are both correct. Current epoch = 1683147857.  The id_token was: {
  "aud": "04b07795-8ddb-461a-bbee-02f9e1bf7b46",
  "iss": "https:/

## Task 4: Create a compute cluster

In [ ]:
from azure.ai.ml.entities import AmlCompute

COMPUTE_NAME = 'trainingcompute'

my_compute = AmlCompute(
    name=COMPUTE_NAME,
    size='Standard_DS2_v2',
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=3600
)
ml_client.compute.begin_create_or_update(my_compute)

## Task 5: Create the job

In [ ]:
from azure.ai.ml import command, Input, Output

TARGET_COLUMN_NAME = 'readmit_status'

# Create the job.
job = command(
    description='Trains hospital readmission model',
    experiment_name='hospital_readmission',
    compute=COMPUTE_NAME,
    inputs=dict(training_data=Input(type='uri_file', path=f'{TRAIN_DATA_NAME}@latest'), 
                target_column_name=TARGET_COLUMN_NAME),
    outputs=dict(model_output=Output(type=AssetTypes.MLFLOW_MODEL)),
    code='src/',
    environment='azureml://registries/azureml/environments/AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu/versions/4',
    command='python train.py ' + 
            '--training_data ${{inputs.training_data}} ' +
            '--target_column_name ${{inputs.target_column_name}} ' +
            '--model_output ${{outputs.model_output}}'
)
job = ml_client.jobs.create_or_update(job)
ml_client.jobs.stream(job.name)

RunId: good_loquat_kpfzrg3f6s
Web View: https://ml.azure.com/runs/good_loquat_kpfzrg3f6s?wsid=/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/resourcegroups/rg-bstollnitz/workspaces/mlws-bstollnitz

Execution Summary
RunId: good_loquat_kpfzrg3f6s
Web View: https://ml.azure.com/runs/good_loquat_kpfzrg3f6s?wsid=/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/resourcegroups/rg-bstollnitz/workspaces/mlws-bstollnitz



## Task 6: Register the model

In [ ]:
from azure.ai.ml.entities import Model

MODEL_NAME = 'hospital_readmission_model'

# Register the model.
model_path = f"azureml://jobs/{job.name}/outputs/model_output"
model = Model(name=MODEL_NAME,
                path=model_path,
                type=AssetTypes.MLFLOW_MODEL)
registered_model = ml_client.models.create_or_update(model)

## Task 7: Deploy the model

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment, ManagedOnlineEndpoint 

ENDPOINT_NAME = 'hospital-readmission-endpoint'
DEPLOYMENT_NAME = 'blue'

# Create the managed online endpoint.
endpoint = ManagedOnlineEndpoint(
    name=ENDPOINT_NAME,
    auth_mode='key',
)
registered_endpoint = ml_client.online_endpoints.begin_create_or_update(
    endpoint)

# Create the managed online deployment.
deployment = ManagedOnlineDeployment(name=DEPLOYMENT_NAME,
                                        endpoint_name=ENDPOINT_NAME,
                                        model=registered_model,
                                        instance_type='Standard_DS4_v2',
                                        instance_count=1)
ml_client.online_deployments.begin_create_or_update(deployment)

# Set deployment traffic to 100%.
registered_endpoint.traffic = {DEPLOYMENT_NAME: 100}
ml_client.online_endpoints.begin_create_or_update(
    registered_endpoint)

Check: endpoint hospital-readmission-endpoint exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored
Creating/updating online deployment blue 

.....................................

AzureCliCredential.get_token failed: Failed to invoke the Azure CLI


CredentialUnavailableError: Failed to invoke the Azure CLI

## Task 8: Invoke the endpoint

In [ ]:
TEST_DATA_PATH="test_data.json"

# Invoke the endpoint.
result = ml_client.online_endpoints.invoke(endpoint_name=ENDPOINT_NAME, request_file=TEST_DATA_PATH)
print(result)

["not readmitted"]
